In [13]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow.keras.layers import Lambda
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import argparse
import os
import math

import sys
import csv
import re
import random
import copy
from skimage.transform import downscale_local_mean
from skimage import io
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Multiply, Input, Dropout, Conv2D, Conv3D, Conv2DTranspose, Conv3DTranspose, MaxPooling2D, MaxPooling3D, UpSampling2D, Dense, Flatten, Activation, Reshape, BatchNormalization, LeakyReLU, PReLU
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import initializers

from __future__ import print_function, division
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, LeakyReLU
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import sys
import numpy as np
from math import log

from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Add
from tensorflow.keras.constraints import min_max_norm

SyntaxError: from __future__ imports must occur at the beginning of the file (<ipython-input-13-c1989247cccf>, line 4)

In [14]:
tf.set_random_seed(2435)

In [15]:
data_root = 'D:/HDD Data/cartoons/'
output_root = 'D:/HDD Data/cartoons'
random.seed()

In [16]:
def load_metadata():
    metadata = []
    with open(data_root + 'metadata.csv', 'r', newline='') as metadata_file:
        reader = csv.reader(metadata_file, quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for row in reader:
            metadatum = row[:2] + [int(n) for n in re.findall(r'\d+', row[2])]
            metadata.append(metadatum)
    return metadata

In [17]:
metadata = load_metadata()
random.shuffle(metadata)
metadata_train = metadata[len(metadata)//10:]
metadata_test = metadata[:len(metadata_train)]
num_data = len(metadata)

classes = set([datum[0] for datum in metadata])
# build a dictionary mapping between name strings and ids
class_to_id = dict((n, i) for i, n in enumerate(classes))
id_to_class = dict((i, n) for i, n in enumerate(classes))
num_classes = len(classes)

In [18]:
def report_epoch_progress(epoch, logs, gen_model, data_test):
    print('epoch', epoch, 'progress report:')
    example = data_test.__getitem__(3)
    latent = example[0]
    print("latent shape: ", latent.shape)
    print("latent: ", latent)
    img = gen_model.predict(latent)
    img = np.squeeze(img, axis=(0,3))
    img *= 255.0
    img = img.astype(int)
    print("output shape: ", img.shape)

    actual = example[1]
    actual = np.squeeze(actual, axis=(0,3))
    #actual = np.expand_dims(actual, axis=3)
    actual *= 255.0
    actual = actual.astype(int)
    print("actual shape: ", actual.shape)


    #latent = np.squeeze(latent, axis=(0,3))
    #latent *= 255.0
    #latent = latent.astype(int)

    #print("INPUT")
    #io.imshow(latent, cmap='gray')
    #io.show()
    print("OUTPUT")
    io.imshow(img)#, cmap='gray')
    io.show()
    print("TARGET")
    io.imshow(actual)#, cmap='gray')
    io.show()

In [19]:
def test_gan(g_input, g_output, d_input, d_output, epochs=2, first_category_only=False, input_method=flat_vector, inputs=None, g_lr=.0005, d_lr=.00001, gan_lr=.0001):
    # we have to reinstantiate the generator and discriminator or the weights won't be reset
    generator = Model(g_input, g_output, name='generator')
    discriminator = Model(d_input, d_output, name='discriminator')
    gan_model = GANModel(generator, discriminator, inputs)
    gan_model.compile(g_lr, d_lr, gan_lr)
    data_train = DataProvider(metadata_train, first_category_only, get_input_vector=input_method)
    data_test = DataProvider(metadata_test, first_category_only, get_input_vector=input_method)
    gan_model.train(epochs, data_train, data_test, verbose=True)
    return gan_model

In [20]:
def flat_vector(img, latent_category):
    return np.random.rand(100)

class DataProvider2():

    metadata = None
    batch_size = 1 # because I get OOM otherwise
    
    def __init__(self, metadata, first_category_only=False, get_input_vector=flat_vector):
        self.metadata = metadata
        self.first_category_only = first_category_only
        self.get_input_vector = get_input_vector
        self.downscale_factor = 64
        self.img_x, self.img_y = 5, 14
    
    def __len__(self):
        return len(self.metadata)
    
    def get_batch(self):
        x, y = self._get_batch(self.batch_size, self.metadata)
        if x is None or y is None:
            raise ValueError("input x or y is none")
        np.swapaxes(x, 0,1)
        return x, y
    
    def upscale(self):
        self.downscale_factor = int(self.downscale_factor / 2)
        self.img_x *= 2
        self.img_y *= 2
    
    # note to self for the future: optimize the batches to use np arrays from the getgo?
    def _get_batch(self, batch_size, metadata):
        img_x, img_y = self.img_x, self.img_y
        batch_x = np.zeros((batch_size, img_x, img_y, 1), dtype=float)
        batch_y = np.zeros((batch_size, img_x, img_y, 1), dtype=float)
        datum_index = random.randint(0, len(metadata)-1)

        batch_x = self.get_input_vector(batch_y, None)
            
        for i in range(batch_size):
            img_scaled = None
            datum_index = random.randint(0, len(metadata)-1)
            j = 0
            while img_scaled is None:
                metadatum = metadata[(datum_index + j) % len(metadata)]
                latent_category = class_to_id[metadatum[0]]
                if latent_category == num_classes-1 or self.first_category_only is False:
                    img_scaled = self.get_image(metadata, (datum_index + j) % len(metadata))
                else:
                    img_scaled = None
                j += 1
            
            # put it in a tensor after downscaling it and padding it
            img_downscaled = downscale_local_mean(img_scaled, (self.downscale_factor, self.downscale_factor, 1))
            # normalize channel values
            if img_downscaled.shape[1] < img_y:
                batch_y[i] = np.pad(img_downscaled, ((0,0), (0, img_y - img_downscaled.shape[1]), (0, 0)), 'maximum')
            else:
                batch_y[i] = img_downscaled[:,:img_y]

            batch_y[i] /= 255.0
            batch_x = self.get_input_vector(batch_y, latent_category)

        return batch_x.reshape(1,-1), batch_y

    # if outside the tolerance range, return None (it's not a valid datum)
    # if to large, crop from both sides to fit
    # if to small, pad with maximum value (white) to fit
    def scale_to_target(self, image, initial_y, target_y, shrink_tolerance, grow_tolerance):
        if(target_y-initial_y > grow_tolerance or initial_y-target_y > shrink_tolerance):
            return None
        elif(initial_y > target_y):
            return image[:target_y]
        else: # initial_y <= target_y
            padding = (target_y-initial_y)//2
            return np.pad(image, ((padding, target_y - initial_y - padding),(0,0),(0,0)), 'maximum')

    def get_image(self, metadata, datum_index):
        x, y = 160*2, 448*2
        metadatum = metadata[datum_index]
        img_raw = io.imread(data_root + 'images/' + metadatum[0] + metadatum[1] + '.png', as_gray=True)
        #if len(img_raw.shape) == 2:
        #    return None
        img_raw = np.expand_dims(img_raw, -1)
        img_scaled = self.scale_to_target(img_raw, int(metadatum[2]), x, 10, 120)
        return img_scaled

In [21]:
from tensorflow.keras.optimizers import SGD


def wasserstein_loss(y_true, y_pred):
    """Calculates the Wasserstein loss for a sample batch.
    The Wasserstein loss function is very simple to calculate. In a standard GAN, the
    discriminator has a sigmoid output, representing the probability that samples are
    real or generated. In Wasserstein GANs, however, the output is linear with no
    activation function! Instead of being constrained to [0, 1], the discriminator wants
    to make the distance between its output for real and generated samples as
    large as possible.
    The most natural way to achieve this is to label generated samples -1 and real
    samples 1, instead of the 0 and 1 used in normal GANs, so that multiplying the
    outputs by the labels will give you the loss immediately.
    Note that the nature of this loss means that it can be (and frequently will be)
    less than 0."""
    return K.mean(y_true * y_pred)


def combined_loss(y_true, y_pred):
    return wasserstein_loss(y_true, y_pred) + tf.keras.losses.binary_crossentropy(y_true, y_pred)


class DCGAN():
    
    def __init__(self, g, d):
        g_inputs, g_outputs, self.mapper = g()
        d_inputs, d_outputs = d()
        
        self.img_rows = 10
        self.img_cols = 28
        self.channels = 1
        self.latent_dim = 100

        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        self.generator = Model(inputs=g_inputs, outputs=g_outputs)
        self.discriminator = Model(inputs=d_inputs, outputs=d_outputs)
        
        # [noise, self.constant_input, self.noise_input]
        gan_inputs = [Input(shape=(100,)), Input(shape=(5, 14, 32)), Input(shape=(5, 14, 32))]
        
        gan_images = self.generator(gan_inputs)
        gan_output = self.discriminator(gan_images)

        self.gan = Model(inputs=gan_inputs, outputs=gan_output)
        self.g_inputs = g_inputs
        self.g_outputs = g_outputs
        self.d_inputs = d_inputs
        self.d_outputs = d_outputs
        
        self.data_provider = DataProvider2(metadata)
        self.stage = 0
    
    def compile(self):

        #optimizer = Adam()
        optimizer = Adam(0.0002, 0.5, clipnorm=1.0)
        self.discriminator.compile(loss=combined_loss,
            optimizer=optimizer)
        
        self.constant_input = K.variable(np.full((1, 5, 14, 32), .5, dtype='float32'))
        self.noise_input = K.variable(np.full((1, 5, 14, 32), 1, dtype='float32'))

        latent = Input(shape=(self.latent_dim,))
        img = self.generator([latent, self.constant_input, self.noise_input])

        # The discriminator takes generated images as input and determines validity
        self.discriminator.trainable = False
        valid = self.discriminator(img)

        # trains the generator to fool the discriminator
        gen_optimizer = Adam(clipnorm=1.0)
        self.gan.compile(loss=combined_loss, optimizer=gen_optimizer)
        self.gan.summary()
        plot_model(self.gan, to_file='gan_model.png')
        plot_model(self.generator, to_file='generator_model.png')
        plot_model(self.discriminator, to_file='discriminator_model.png')
        
    def add_block(self, block):
        # params needed: block input, n_input (for the block), mapper
        g_input = self.g_outputs
        d_output = self.discriminator
        d_inputs = self.d_inputs
        n_input = self.g_inputs[2]
        mapper = UpSampling2D()(self.mapper)
        new_g_output, discriminator = block(g_input, d_output, d_inputs, n_input, mapper)
        self.g_outputs = new_g_output
        self.generator = Model(inputs=self.g_inputs, outputs=self.g_outputs)
        self.discriminator = discriminator
        
        self.generator.summary()
        self.discriminator.summary()
        
        gan_inputs = [Input(shape=(100,)), Input(shape=(5, 14, 32)), Input(shape=(5, 14, 32))]
        gan_images = self.generator(gan_inputs)
        gan_output = self.discriminator(gan_images)

        self.gan = Model(inputs=gan_inputs, outputs=gan_output)
        self.compile() # TODO: this probably resets learning params and we probably don't want it to do that
        self.data_provider.upscale()
        self.stage += 1
    
    def train(self, epochs, batch_size=128, save_interval=1):

        data_provider = self.data_provider
        
        # Adversarial ground truths
        # for whatever reason, valid=zeros is supposed to help convergence
        # and noise is supposed to help not overtrain, too
        # valid=ones is necessary for wasserstein
        valid = np.random.uniform(.9, .1, (batch_size, 1))
        fake = np.random.uniform(-.9, 1.0, (batch_size, 1))
        #valid = np.zeros((batch_size, 1))
        #fake = np.ones((batch_size, 1))
        
        steps_per_epoch = 100
        
        for epoch in range(epochs):

            d_loss = [10.0, 10.0]
            g_loss = 10.0
            d_loss_threshold = .7
            g_loss_threshold = .75
            steps = 0
            while steps < steps_per_epoch:
                
                # train discriminator to threshold loss or end of epoch, whichever first
                #self.discriminator.trainable = True    
                while True:
                    if steps >= steps_per_epoch:
                        break
                    steps += 1
                    
                    imgs = data_provider.get_batch()[1]
                    
                    # Sample noise and generate a batch of new images
                    noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                    gen_imgs = self.generator.predict([noise, self.constant_input, self.noise_input], steps=1)
                    
                    # Train the discriminator (real classified as ones and generated as zeros)
                    d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                    d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                    # TODO: use shannon entropy instead to discourage counting everythign as valid/invalid?
                    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) # TODO: get it to be an integer aggregating all values (what are they?)

                    #print(d_loss)
                    #if d_loss[0]*log(d_loss[0]) + d_loss[1]*log(d_loss[1]) < d_loss_threshold:
                    #    break

                    if d_loss < d_loss_threshold:
                        break
                # train generator to threshold loss or end of epoch, whichever first
                # halt training on the discriminator
                self.discriminator.trainable = False # does this reset every training step?  where is it enabled again?
                while True:
                    if steps >= steps_per_epoch:
                        break
                    print('training generator')
                    steps += 1
                    # get a new batch of noise
                    noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                    # Train the generator (wants discriminator to mistake images as real)
                    g_loss = self.gan.train_on_batch([noise, self.constant_input, self.noise_input], valid)

                    if g_loss < g_loss_threshold:
                        break

            print ("%d [D loss: %f] [G loss: %f]" % (epoch, d_loss, g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)


    def save_imgs(self, epoch):
        r, c = 1, 1
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        gen_img = self.generator.predict([noise, self.constant_input, self.noise_input], steps=1)[0]

        # Rescale images 0 - 1
        gen_img = 0.5 * gen_img + 0.5

        gen_img = np.squeeze(gen_img, -1)
        io.imshow(gen_img)
        plt.savefig("images/mnist_%d_%d.png" % (self.stage, epoch))
        plt.close()

In [22]:
from tensorflow.keras.layers import Layer


def normalize_data_format(value):
    if value is None:
        value = 'channels_last'
    data_format = value.lower()
    if data_format not in {'channels_first', 'channels_last'}:
        raise ValueError('The `data_format` argument must be one of '
                         '"channels_first", "channels_last". Received: ' +
                         str(value))
    return data_format


class AdaIN(Layer):
    def __init__(self, data_format=None, eps=1e-7, **kwargs):
        super(AdaIN, self).__init__(**kwargs)
        self.data_format = normalize_data_format(data_format)
        #self.data_format = data_format
        self.spatial_axis = [1, 2] if self.data_format == 'channels_last' else [2, 3]
        self.eps = eps

    def compute_output_shape(self, input_shape):
        return input_shape[0]

    def call(self, inputs):
        image = inputs[0]
        if len(inputs) == 2:
            style = inputs[1]
            style_mean, style_var = tf.nn.moments(style, self.spatial_axis, keep_dims=True)
        else:
            style_mean = tf.expand_dims(K.expand_dims(inputs[1], self.spatial_axis[0]), self.spatial_axis[1])
            style_var = tf.expand_dims(K.expand_dims(inputs[2], self.spatial_axis[0]), self.spatial_axis[1])
        image_mean, image_var = tf.nn.moments(image, self.spatial_axis, keep_dims=True)
        out = tf.nn.batch_normalization(image, image_mean,
                                         image_var, style_mean,
                                         tf.sqrt(style_var), self.eps)
        return out


In [23]:
conv_weight_constraint = .01 # this constraint appears by empirical testing to be the most responsible for keeping the output range from blowing up
noise_weight_constraint = .01 # if this is set high the output doesn't change as much (probably because the noise weights dominate)
mapper_weight_constraint = 1000.0 # set this really high as it didn't seem to affect the output
gaussian_noise_mean = .5

def build_mapper():
    m_input = Input(shape=(100,), name='m_input')
    layer = Dense(128, kernel_constraint=min_max_norm(-mapper_weight_constraint, mapper_weight_constraint))(m_input)
    #layer = BatchNormalization(momentum=.9)(layer)
    #layer = Dropout(.3)(layer)
    layer = LeakyReLU(.25)(layer)
    
    layer = Dense(128, kernel_constraint=min_max_norm(-mapper_weight_constraint, mapper_weight_constraint))(layer)
    #layer = BatchNormalization(momentum=.9)(layer)
    #layer = Dropout(.3)(layer)
    layer = LeakyReLU(.25)(layer)
    
    layer = Dense(128, kernel_constraint=min_max_norm(-mapper_weight_constraint, mapper_weight_constraint))(layer)
    #layer = BatchNormalization(momentum=.9)(layer)
    #layer = Dropout(.3)(layer)
    layer = LeakyReLU(.25)(layer)

    layer = Dense(2240, kernel_constraint=min_max_norm(-mapper_weight_constraint, mapper_weight_constraint))(layer)
    layer = LeakyReLU(.25)(layer)
    
    m_output = Reshape((5, 14, 32), name='mapper_reshape')(layer)
    #mapper_model = Model(inputs=m_input, outputs=m_output)
    #plot_model(mapper_model, to_file='mapper_model.png')
    return m_input, m_output


def build_generator():
        
        mapper_input, mapper = build_mapper()
        #g_input = Input(tensor=K.variable(np.random.uniform(size=(1, 5, 14, 32)), name='constant_input'))
        g_input = Input(shape=(5, 14, 32), name='constant_input')
        layer = g_input
        layer = Conv2DTranspose(32, 3, padding='same', kernel_constraint=min_max_norm(-conv_weight_constraint, conv_weight_constraint, axis=[0,1,2]))(layer)
        layer = LeakyReLU(.25)(layer)
        # add AdaIN function with noise and mapper inputs
        #n_input = Input(tensor=K.variable(tf.zeros(shape=layer.shape)), name='noise_input')
        n_input = Input(shape=(5, 14, 32), name='noise_input')
        u_noise = GaussianNoise(gaussian_noise_mean)(n_input)
        u_noise = Flatten()(u_noise) # is this necessary?
        noise = Dense(2240, kernel_constraint=min_max_norm(-noise_weight_constraint, noise_weight_constraint))(u_noise)
        noise = Reshape((5, 14, 32))(noise)
        layer = Add()([layer, noise])
        layer = AdaIN()([layer, mapper])
        #layer = BatchNormalization(momentum=.9)(layer)
        # the above again
        layer = Conv2DTranspose(32, 3, padding='same', kernel_constraint=min_max_norm(-conv_weight_constraint, conv_weight_constraint, axis=[0,1,2]))(layer)
        layer = LeakyReLU(.25)(layer)
        u_noise2 = GaussianNoise(gaussian_noise_mean)(n_input)
        u_noise2 = Flatten()(u_noise2) # is this necessary?
        noise2 = Dense(2240, kernel_constraint=min_max_norm(-noise_weight_constraint, noise_weight_constraint))(u_noise2)
        noise2 = Reshape((5, 14, 32))(noise2)
        #layer = Add()([layer, noise2])
        #layer = AdaIN()([layer, mapper])
        #layer = BatchNormalization(momentum=.9)(layer)
        #layer = UpSampling2D()(layer)
        # and repeat
        # add AdaIN function with noise and mapper inputs
        #layer = Conv2DTranspose(32, 3, padding='same')(layer)
        #layer = LeakyReLU(.25)(layer)
        #layer = GaussianNoise(.5)(layer)
        #layer = AdaIN()([layer, mapper])
        #layer = BatchNormalization(momentum=.9)(layer)
        # the above again
        #layer = Conv2DTranspose(32, 3, padding='same')(layer)
        #layer = LeakyReLU(.25)(layer)
        #layer = GaussianNoise(.5)(layer)
        #layer = AdaIN()([layer, mapper])
        #layer = BatchNormalization(momentum=.9)(layer)
        #layer = UpSampling2D()(layer)
        
        layer = Conv2DTranspose(128, 3, padding='same', kernel_constraint=min_max_norm(-conv_weight_constraint, conv_weight_constraint, axis=[0,1,2]))(layer)
        layer = LeakyReLU(.25)(layer)
        layer = Conv2DTranspose(1, 2, padding='same', kernel_constraint=min_max_norm(-conv_weight_constraint, conv_weight_constraint, axis=[0,1,2]))(layer)
        # no (nonlinear) activation necessary for wasserstein
        #layer = Activation('sigmoid')(layer)
        print('generator output shape: ', layer.shape)

        g_output = layer
        
        g_inputs = [mapper_input, g_input, n_input]
        g_outputs = g_output
        
        g = Model(inputs=g_inputs, outputs=g_output)
        plot_model(g, to_file='model.png')
        print("plotted model")
        
        return g_inputs, g_outputs, mapper


# TODO: may want to have multiple AdaIn per block as in the original paper, but let's see if this works first
def add_block(g_input, d_output, d_inputs, n_input, mapper):
    
    # generator
    layer = g_input
    layer = UpSampling2D()(layer)
    layer = Conv2DTranspose(32, 3, padding='same', kernel_constraint=min_max_norm(-conv_weight_constraint, conv_weight_constraint, axis=[0,1,2]))(layer)
    layer = LeakyReLU(.25)(layer)
    u_noise2 = UpSampling2D()(n_input)
    u_noise2 = GaussianNoise(gaussian_noise_mean)(u_noise2)
    u_noise2 = Flatten()(u_noise2) # is this necessary?
    noise2 = Dense(8960, kernel_constraint=min_max_norm(-noise_weight_constraint, noise_weight_constraint))(u_noise2)
    noise2 = Reshape((10, 28, 32))(noise2)
    layer = Add()([layer, noise2])
    layer = AdaIN()([layer, mapper])
    # may also be issues with the output layer here
    layer = Conv2DTranspose(128, 3, padding='same', kernel_constraint=min_max_norm(-conv_weight_constraint, conv_weight_constraint, axis=[0,1,2]))(layer)
    layer = LeakyReLU(.25)(layer)
    g_output = Conv2DTranspose(1, 2, padding='same', kernel_constraint=min_max_norm(-conv_weight_constraint, conv_weight_constraint, axis=[0,1,2]))(layer)
    
    # discriminator
    d_input = Input(shape=(10, 28, 1))
    dlayer = Conv2D(filters=32, kernel_size=(4, 4), strides=1, padding='same')(d_input)
    dlayer = BatchNormalization()(dlayer)
    dlayer = LeakyReLU(.25)(dlayer)
    dlayer = Dropout(.3)(dlayer)
    dlayer = Conv2D(filters=32, kernel_size=(4, 4), strides=1, padding='same')(dlayer)
    dlayer = LeakyReLU(.25)(dlayer)
    dlayer = Dropout(.3)(dlayer)
    dlayer = Conv2D(filters=1, kernel_size=(4, 4), strides=2, padding='same')(dlayer)
    submodel = Model(inputs=d_input, outputs=dlayer)
    #print(d_output.layers[0].input_shape)
    
    #gan_images = self.generator(gan_inputs)
    #gan_output = self.discriminator(gan_images)
    #self.gan = Model(inputs=gan_inputs, outputs=gan_output)
    partial_output = submodel(d_input)
    full_output = d_output(partial_output)
    tmp_model = Model(inputs=d_input, outputs=full_output)
    #print(tmp_model.layers[-1].output_shape)
    #print(dlayer.shape)
    #d_output = d_output(tmp_model)
    d_output = tmp_model
    return g_output, d_output


def build_discriminator():
    d_input = Input(shape=(5, 14, 1))
    layer = Conv2D(filters=32, kernel_size=(4, 4), strides=4, padding='same')(d_input)
    layer = BatchNormalization()(layer)
    layer = LeakyReLU(.25)(layer)
    layer = Dropout(.3)(layer)
    layer = Conv2D(filters=32, kernel_size=(4, 4), strides=4, padding='same')(layer)
    layer = LeakyReLU(.25)(layer)
    layer = Dropout(.3)(layer)
    layer = Flatten()(layer)
    layer = Dense(128, name='problem_layer')(layer)
    layer = Activation('sigmoid')(layer)
    layer = Dense(1)(layer)
    d_output = Activation('sigmoid')(layer)

    return d_input, d_output

generator = build_generator
discriminator = build_discriminator
dcgan = DCGAN(generator, discriminator)
dcgan.compile()
dcgan.train(epochs=2, batch_size=1, save_interval=1)
dcgan.add_block(add_block)
dcgan.train(epochs=2, batch_size=1, save_interval=1)

generator output shape:  (?, 5, 14, 1)
plotted model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           (None, 5, 14, 32)    0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 5, 14, 32)    0                                            
__________________________________________________________________________________________________
model_9 (Model)                 (None, 5, 14, 1)     5410753     input_12[0][0]                   
                                                        

0 [D loss: 0.917079] [G loss: 10.000000]


1 [D loss: 0.917357] [G loss: 10.000000]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
m_input (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 128)          12928       m_input[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_13 (LeakyReLU)      (None, 128)          0           dense_8[0][0]                    
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 128)          16512       leaky_re_lu_13[0][0]             
____________________________________________________________________

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 5, 14, 32)    0                                            
__________________________________________________________________________________________________
input_19 (InputLayer)           (None, 5, 14, 32)    0                                            
__________________________________________________________________________________________________
model_14 (Model)                (None, 10, 28, 1)    85739138    input_17[0][0]                   
                                                                 input_18[0][0]                   
          

0 [D loss: 0.895868] [G loss: 10.000000]


1 [D loss: 0.898664] [G loss: 10.000000]
